In [ ]:
from pysph.solver.utils import load
from pysph.solver.utils import iter_output
import h5py

In [ ]:
path = "sine_velocity_profile_output/sine_velocity_profile_00000.hdf5"
# data = load(path)

In [ ]:
# data = h5py.File(path, 'r')

## Energy Spectrum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from energy_spectrum import compute_scalar_energy_spectrum, compute_energy_spectrum # type: ignore

In [ ]:
twopi = 2 * np.pi
sin, cos = np.sin, np.cos

### Single Frequency

#### 1D

In [ ]:
sr = 51
_x = np.arange(1/(2*sr), 1., 1. / sr)
x = _x
u = - cos(twopi * x)
U0 = 1.

# When
EK_U, EK_V, EK_W, u_spectrum, v_spectrum, w_spectrum =\
    compute_energy_spectrum(
        u=u, v=None, w=None, U0=U0, debug=True
    )
plt.stem(EK_U);
plt.show()

k, Ek = compute_scalar_energy_spectrum(EK_U)
plt.stem(Ek)
plt.grid()
print(f"Max (us, vs, ws) = {np.max(u_spectrum)}, {np.max(v_spectrum)}, {np.max(w_spectrum)}")
max_idx = np.argmax(EK_U)
max_idx = np.unravel_index(max_idx, EK_U.shape)
print(f"Max (Eu, Ev, Ew) = {np.max(EK_U)}, {np.max(EK_V)}, {np.max(EK_W)} at [{max_idx}] || (0.5^2)/2")
print(f"Max Ek: {np.max(Ek)} at [{np.argmax(Ek)}] || (0.125*2)/2 because 2 peaks")


#### 2D

In [ ]:
sr = 19
_x = np.arange(1/(2*sr), 1., 1. / sr)
x, y = np.meshgrid(_x, _x)
# u = sin(twopi * x) + sin(2 * twopi * y)
# v = u
u = - cos(twopi * x) * sin(twopi * y)
v = sin(twopi * x) * cos(twopi * y)
U0 = 1.

# When
EK_U, EK_V, EK_W, u_spectrum, v_spectrum, w_spectrum =\
    compute_energy_spectrum(
        u=u, v=v, w=None, U0=U0, debug=True
    )
plt.imshow(EK_U)
plt.colorbar()
plt.axis('equal')
plt.gca().invert_yaxis()
plt.title("EK_U")
plt.show()

k, Ek = compute_scalar_energy_spectrum(EK_U, EK_V, ord=np.inf)
plt.stem(Ek)
plt.grid()
plt.title("Ek")
max_idx = np.argmax(EK_U)
max_idx = np.unravel_index(max_idx, EK_U.shape)
print(f"Max (us, vs, ws) = {np.max(u_spectrum)}, {np.max(v_spectrum)}, {np.max(w_spectrum)}")
print(f"Max (Eu, Ev, Ew) = {np.max(EK_U)}, {np.max(EK_V)}, {np.max(EK_W)} at [{max_idx}] || (0.25^2)/2")
print(f"Max Ek: {np.max(Ek)} at [{np.argmax(Ek)}] || (0.03125*8)/2 beacuse 8 peaks")

#### 3D

In [ ]:
x, y, z = np.meshgrid(_x, _x, _x)
u = - cos(twopi * x) * sin(twopi * y) * sin(twopi * z)
v = sin(twopi * x) * cos(twopi * y) * sin(twopi * z)
w = sin(twopi * x) * sin(twopi * y) * cos(twopi * z)

EK_U, EK_V, EK_W, u_spectrum, v_spectrum, w_spectrum =\
    compute_energy_spectrum(
        u=u, v=v, w=w, U0=U0, debug=True
    )

k, Ek = compute_scalar_energy_spectrum(EK_U, EK_V, EK_W, ord=2)
plt.stem(Ek)
plt.grid()
max_idx = np.argmax(EK_U)
max_idx = np.unravel_index(max_idx, EK_U.shape)
print(f"Max (us, vs, ws) = {round(np.max(u_spectrum), 8)}, {round(np.max(v_spectrum), 8)}, {round(np.max(w_spectrum), 8)}")
print(f"Max (Eu, Ev, Ew) = {round(np.max(EK_U), 8)}, {round(np.max(EK_V), 8)}, {round(np.max(EK_W), 8)} at [{max_idx}] || (0.125^2)/2")
print(f"Max Ek: {np.max(Ek)} at [{np.argmax(Ek)}] || (0.0078125*24)/2 because 24 peaks")

### Summary

```
1D
* Max (us, vs, ws) = 0.5, 0.0, 0.0 || Max(u)/2^1
* Max (Eu, Ev, Ew) = 0.125, 0.0, 0.0 at [(14,)] || (0.5^2)/2
* Max Ek: 0.125 at [1] || (0.125*2)/2 because 2 peaks

2D
* Max (us, vs, ws) = 0.25, 0.25, 0.0 || Max(u)/2^2
* Max (Eu, Ev, Ew) = 0.03125, 0.03125, 0.0 at [(14, 14)] || (0.25^2)/2
* Max Ek: 0.125 || (0.03125*8)/2 beacuse 8 peaks

3D
* Max (us, vs, ws) = 0.125, 0.125, 0.125 || Max(u)/2^3
* Max (Eu, Ev, Ew) = 0.0078125, 0.0078125, 0.0078125 at [(17, 17, 17)] || (0.125^2)/2
* Max Ek: 0.09375 || (0.0078125*24)/2 because 24 peaks 
    at [1] if ord=np.inf, at [2] if ord=2

* Max (us, vs, ws) = 0.125, 0.625, 0.125 || Max(u)/2^3
* Max (Eu, Ev, Ew) = 0.0078125, 0.1953125, 0.0078125 at [(14, 14, 14)] || (0.125^2)/2
* Max Ek: 0.84375 || (0.0078125*24)/2 because 24 peaks
```

## Multiple Frequencies

Velocity is squared to get the energy spectrum.
Therefore, the slope of the energy spectrum will be 2 times the slope of the velocity spectrum.

In [ ]:
# Machine precision
EPS = np.finfo(float).eps


### 1D

In [ ]:
sr = 51
_x = np.arange(1/(2*sr), 1., 1. / sr)
x = _x
num_freqs = 17
u = np.zeros_like(x)
slope_vel = -2
for i in range(1, num_freqs + 1):
    u += -cos(i * twopi * x) * i**(slope_vel)
U0 = 1.

# When
EK_U, EK_V, EK_W, u_spectrum, v_spectrum, w_spectrum =\
    compute_energy_spectrum(
        u=u, v=None, w=None, U0=U0, debug=True
    )

k, Ek = compute_scalar_energy_spectrum(EK_U)

plt.figure(figsize=(16, 5))
plt.subplot(1, 3, 1)
plt.stem(EK_U);
plt.title("EK_U")

plt.subplot(1, 3, 2)
plt.stem(Ek)
plt.grid()
plt.title("E(k)")

# Get all datapoints of E(k) greater than 1e-8
tol = 1e-8
k_cacl = k[Ek > tol]
Ek_cacl = Ek[Ek > tol]

k_calc_log = np.log10(k_cacl)
Ek_calc_log = np.log10(Ek_cacl)
slope, intercept, r_value, p_value, std_err = linregress(k_calc_log, Ek_calc_log)

# Calculate fit
k_fit = k_cacl
Ek_fit = 10**(slope * np.log10(k_fit) + intercept)

plt.subplot(1, 3, 3)
plt.loglog(k, Ek, 'ko', label=r'$E(k)$')
plt.loglog(k, 1e-3 * (EPS+k)**(-5/3), 'r--', label=r'$k^{-5/3}$')
plt.loglog(k_fit, Ek_fit, 'b-', label=f"Fit: $E(k) = k^{{{slope:.2f}}}$")
# plt.ylim(1e-8, 1e0)
plt.legend()
plt.grid()
plt.title("Log-log plot of E(k)")


print(f"Slope of velocity spectrum: {slope_vel}")
print(f"slope = {slope}, intercept = {intercept}, r_value = {r_value}, p_value = {p_value}, std_err = {std_err}")

In [ ]:
# print all values of Ek upto 4 decimal places without using round()
np.set_printoptions(formatter={'float': lambda x: "{0:0.6f}".format(x)})
print(f"Ek = {Ek}")

In [ ]:
a = np.arange(1, num_freqs + 1, dtype=float)
a = a**(2*slope_vel)/8
b = np.insert(a, 0, 0)
b = np.append(b, np.zeros(len(k) - len(b)))
print(b)
plt.plot(k, np.abs(Ek - b), 'k--')

### 2D

In [ ]:
sr = 33
_x = np.arange(1/(2*sr), 1., 1. / sr)
x, y = np.meshgrid(_x, _x)
num_freqs = 13
u = np.zeros_like(x)
v = np.zeros_like(y)
slope_vel = -1
for i in range(1, num_freqs + 1):
    u += -cos(i * twopi * x) * sin(i * twopi * y) * i**(slope_vel)
    v += sin(i * twopi * x) * cos(i * twopi * y) * i**(slope_vel)
U0 = 1.

# When
EK_U, EK_V, EK_W, u_spectrum, v_spectrum, w_spectrum =\
    compute_energy_spectrum(
        u=u, v=v, w=None, U0=U0, debug=True
    )

k, Ek = compute_scalar_energy_spectrum(EK_U, EK_V, ord=np.inf)

plt.figure(figsize=(16, 5))
plt.subplot(1, 3, 1)
plt.imshow(EK_U)
plt.colorbar()
plt.axis('equal')
plt.gca().invert_yaxis()
plt.title("EK_U")

plt.subplot(1, 3, 2)
plt.stem(Ek)
plt.grid()
plt.title("Ek")

# Get all datapoints of E(k) greater than 1e-8 and k between 0 and len(k)/2
tol = 1e-8
cond_len = k < len(k)/2 - 1
cond_tol = Ek > tol
cond = cond_len & cond_tol
k_cacl = k[cond]
Ek_cacl = Ek[cond]

k_calc_log = np.log10(k_cacl)
Ek_calc_log = np.log10(Ek_cacl)
slope, intercept, r_value, p_value, std_err = linregress(k_calc_log, Ek_calc_log)

# Calculate fit
k_fit = k_cacl
Ek_fit = 10**(slope * np.log10(k_fit) + intercept)

plt.subplot(1, 3, 3)
plt.loglog(k, Ek, 'k.', label=r'$E(k)$')
plt.loglog(k, 1e-3 * (EPS+k)**(-5/3), 'r--', label=r'$k^{-5/3}$')
plt.loglog(k_fit, Ek_fit, 'b-', label=f"Fit: $E(k) = k^{{{slope:.2f}}}$")
plt.ylim(1e-8, 1e0)
plt.legend()
plt.grid()
plt.title("Log-log plot of E(k)")


print(f"Slope of velocity spectrum: {slope_vel}")
print(f"Slope = {slope}, intercept = {intercept}, r_value = {r_value}, p_value = {p_value}, std_err = {std_err}")

In [ ]:
print(f"{Ek = }")

In [ ]:
a = np.arange(1, num_freqs + 1, dtype=float)
a = a**(2*slope_vel)/8
b = np.insert(a, 0, 0)
b = np.append(b, np.zeros(len(k) - len(b)))
print(f"{b = }")
plt.plot(k, np.abs(Ek - b), 'k--')

### 3D

In [ ]:
sr = 33
_x = np.arange(1/(2*sr), 1., 1. / sr)
x, y, z = np.meshgrid(_x, _x, _x)
num_freqs = 15
u, v, w = np.zeros_like(x), np.zeros_like(y), np.zeros_like(z)
slope_vel = -1
for i in range(1, num_freqs + 1):
    u += -cos(i * twopi * x) * sin(i * twopi * y) * sin(i * twopi * z) * i**(slope_vel)
    v += sin(i * twopi * x) * cos(i * twopi * y) * sin(i * twopi * z) * i**(slope_vel)
    w += - sin(i * twopi * x) * sin(i * twopi * y) * cos(i * twopi * z) * i**(slope_vel)

EK_U, EK_V, EK_W, u_spectrum, v_spectrum, w_spectrum =\
    compute_energy_spectrum(
        u=u, v=v, w=w, U0=U0, debug=True
    )

k, Ek = compute_scalar_energy_spectrum(EK_U, EK_V, EK_W, ord=np.inf)

plt.figure(figsize=(16, 5))
plt.subplot(1, 2, 1)
plt.stem(Ek)
plt.grid()
plt.title("Ek")


# Get all datapoints of E(k) greater than 1e-8 and k between 0 and len(k)/2
tol = 1e-8
cond_len = k < len(k)/2 - 1
cond_tol = Ek > tol
cond = cond_len & cond_tol
k_cacl = k[cond]
Ek_cacl = Ek[cond]

k_calc_log = np.log10(k_cacl)
Ek_calc_log = np.log10(Ek_cacl)
slope, intercept, r_value, p_value, std_err = linregress(k_calc_log, Ek_calc_log)

# Calculate fit
k_fit = k_cacl
Ek_fit = 10**(slope * np.log10(k_fit) + intercept)

plt.subplot(1, 2, 2)
plt.loglog(k, Ek, 'k.', label=r'$E(k)$')
plt.loglog(k, 1e-3 * (EPS+k)**(-5/3), 'r--', label=r'$k^{-5/3}$')
plt.loglog(k_fit, Ek_fit, 'b-', label=f"Fit: $E(k) = k^{{{slope:.2f}}}$")
plt.ylim(1e-8, 1e0)
plt.legend()
plt.grid()
plt.title("Log-log plot of E(k)")


print(f"Slope of velocity spectrum: {slope_vel}")
print(f"Slope = {slope}, intercept = {intercept}, r_value = {r_value}, p_value = {p_value}, std_err = {std_err}")

In [ ]:
print(f"{Ek = }")

In [ ]:
a = np.arange(1, num_freqs + 1, dtype=float)
a = 3*a**(2*slope_vel)/32
b = np.insert(a, 0, 0)
b = np.append(b, np.zeros(len(k) - len(b)))
print(f"{b = }")
plt.plot(k, np.abs(Ek - b), 'k--')

## Effect of sampling frequency and number of samples on Fourier transform

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual

### Aliasing

In [2]:
def calc_perceived_freq(sig_freq, sample_freq):
    nyquist_freq = sample_freq / 2
    perceived_freq = np.abs(
        sig_freq - sample_freq*np.round(sig_freq / sample_freq)
    )
    return nyquist_freq, perceived_freq
calc_perceived_freq(sig_freq=4, sample_freq=10)

(5.0, 4.0)

In [57]:
def study_fft_relationship_aliasing(L=1., sig_freq=2, sample_freq=11,):
    twopi = 2 * np.pi
    sin, cos = np.sin, np.cos

    # Create the time array
    t = np.linspace(0, L, 1000)
    t_sample = np.arange(1/(2*sample_freq), L, L / sample_freq)
    
    y_exact = np.zeros_like(t)
    y_sample = np.zeros_like(t_sample)
    
    y_exact += cos(sig_freq * twopi * t)
    y_sample += cos(sig_freq * twopi * t_sample)

    # Compute the FFT
    Y = np.fft.fft(y_sample)/(sample_freq*0.5)
    Y = np.abs(Y)

    # Reconstruct the signal from the frequencies present in the FFT
    y_recon = np.zeros_like(t)
    for i in range(len(Y)//2 + 1):
        y_recon += Y[i] * cos(i * twopi * t)
    
    # Calculate the perceived frequency
    nyquist_freq, perceived_freq = calc_perceived_freq(sig_freq, sample_freq)

    if sig_freq >= nyquist_freq:
        title = "Aliased Reconstruction"
    elif np.abs(np.round(sig_freq) - sig_freq) < 1e-5:
        title = "Perfect Reconstruction"
    else:
        title = "No Aliasing"
    if sig_freq == sample_freq:
        title = 'DC Signal'
        
    plt.figure(figsize=(18, 5))
    plt.subplot(1, 2, 1)
    plt.plot(t, y_exact, label=f"exact, $f_{{sig}} = {sig_freq}$")
    plt.plot(t_sample, y_sample, 'k.', markersize=10, label="sampled")
    plt.plot(
        t, y_recon,
        label=f"reconstructed, $f_{{recon}} = {perceived_freq:.2f}$"
    )
    plt.xlabel("t")
    plt.ylabel("y")
    plt.grid()
    plt.legend()
    plt.title(title)

    plt.subplot(1, 2, 2)
    plt.stem(Y)
    plt.xlim(0, sample_freq//2 + 1)
    plt.xlabel("frequency")
    plt.ylabel("Amplitude")
    plt.grid();
    # plt.show()

In [58]:
# study_fft_relationship_aliasing(L=1., sample_freq=51, sig_freq=2)

In [60]:
interact(
    study_fft_relationship_aliasing, L=(0.1, 5., 0.1),
    sample_freq=(10, 501, 1), sig_freq=(1, 20, 0.01))

interactive(children=(FloatSlider(value=1.0, description='L', max=5.0, min=0.1), FloatSlider(value=2.0, descri…

<function __main__.study_fft_relationship_aliasing(L=1.0, sig_freq=2, sample_freq=11)>

### Aliasing with Interpolation

In [63]:
def interpolator(y_exact, t_exact, t_sample, kind='linear', interp_sample=None):
    """
        Interpolate the signal y_exact(t_exact) at the times t_sample, using
        the interpolation method `kind` with `interp_sample` points.
    """
    y_sample = np.interp(t_sample, t_exact, y_exact)
    return y_sample


In [91]:
def study_fft_relationship_interp(L=1., sig_freq=2, sample_freq=11,):
    twopi = 2 * np.pi
    sin, cos = np.sin, np.cos

    # Create the time array
    t = np.linspace(0, L, 100)
    t_sample = np.arange(1/(2*sample_freq), L, L / sample_freq)
    
    y_exact = cos(sig_freq * twopi * t)
    y_interp = interpolator(y_exact=y_exact, t_exact=t, t_sample=t_sample)
    y_sample = cos(sig_freq * twopi * t_sample)

    y_interp_error = np.abs(y_interp - y_sample)

    # Compute the FFT
    Y_interp = np.fft.fft(y_interp)/(sample_freq*0.5)
    Y_interp = np.abs(Y_interp)
    Y_sample = np.fft.fft(y_sample)/(sample_freq*0.5)
    Y_sample = np.abs(Y_sample)

    Y_interp_error = np.abs(Y_interp - Y_sample)

    # Reconstruct the signal from the frequencies present in the FFT
    y_recon_interp = np.zeros_like(t)
    for i in range(len(Y_interp)//2 + 1):
        y_recon_interp += Y_interp[i] * cos(i * twopi * t)
    
    # Calculate the perceived frequency
    nyquist_freq, perceived_freq = calc_perceived_freq(sig_freq, sample_freq)

    if sig_freq >= nyquist_freq:
        title = "Aliased Reconstruction"
    elif np.abs(np.round(sig_freq) - sig_freq) < 1e-5:
        title = "Perfect Reconstruction"
    else:
        title = "No Aliasing"
    if sig_freq == sample_freq:
        title = 'DC Signal'
        
    plt.figure(figsize=(16, 8))
    plt.subplot(2, 2, 1)
    plt.plot(t, y_exact, label=f"exact, $f_{{sig}} = {sig_freq}$")
    plt.plot(t_sample, y_interp, 'k.', markersize=10, label="sampled")
    plt.plot(
        t, y_recon_interp,
        label=f"reconstructed, $f_{{recon}} = {perceived_freq:.2f}$"
    )
    plt.xlabel("t")
    plt.ylabel("y")
    plt.grid()
    plt.legend()
    plt.title(title)

    plt.subplot(2, 2, 2)
    plt.stem(Y_interp)
    plt.xlim(0, sample_freq//2 + 1)
    plt.ylabel("Amplitude")
    plt.grid()
    plt.title("FFT of Interpolated Signal")

    plt.subplot(2, 2, 3)
    plt.plot(t_sample, y_interp_error, 'ko', label="interpolation error")
    plt.plot(t_sample, Y_interp_error, 'ro', label="FFT error")
    plt.grid()
    plt.legend()

    plt.subplot(2, 2, 4)
    plt.stem(Y_sample)
    plt.xlim(0, sample_freq//2 + 1)
    plt.xlabel("frequency")
    plt.ylabel("Amplitude")
    plt.grid()
    plt.title("FFT of Sampled Signal")

In [92]:
# study_fft_relationship_interp()

In [93]:
interact(
    study_fft_relationship_interp, L=(0.1, 5., 0.1),
    sample_freq=(10, 501, 1), sig_freq=(1, 20, 1))

interactive(children=(FloatSlider(value=1.0, description='L', max=5.0, min=0.1), IntSlider(value=2, descriptio…

<function __main__.study_fft_relationship_interp(L=1.0, sig_freq=2, sample_freq=11)>

### Multiple frequencies

In [ ]:
def study_fft_relationship_multi_freq(L=1., num_samples=51, num_freqs=2):
    twopi = 2 * np.pi
    sin, cos = np.sin, np.cos

    # Create the time array
    t = np.linspace(0, L, 1000)
    t_sample = np.arange(1/(2*num_samples), L, L / num_samples)
    y_sample = np.zeros_like(t_sample)
    y_exact = np.zeros_like(t)
    for i in range(1, num_freqs + 1):
        y_sample += cos(i * twopi * t_sample)
        y_exact += cos(i * twopi * t)

    # Compute the FFT
    Y = np.fft.fft(y_sample)/(num_samples*0.5)
    Y = np.abs(Y)

    # Reconstruct the signal from the frequencies present in the FFT
    y_recon = np.zeros_like(t)
    for i in range(len(Y)//2):
        y_recon += Y[i] * cos(i * twopi * t)

    plt.figure(figsize=(16, 5))
    plt.subplot(1, 2, 1)
    plt.plot(t, y_exact, label="exact")
    plt.plot(t_sample, y_sample, 'k.', markersize=10, label="sampled")
    plt.plot(t, y_recon, label="reconstructed")
    plt.xlabel("t")
    plt.ylabel("y")
    plt.grid()
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.stem(Y)
    plt.xlim(0, num_samples//2)
    plt.xlabel("frequency")
    plt.ylabel("Amplitude")
    plt.grid()

In [ ]:
study_fft_relationship_multi_freq(L=2, num_samples=11, num_freqs=2)

In [ ]:
interact(study_fft_relationship_multi_freq, L=(1, 5, 0.1),num_samples=(10, 100, 1), num_freqs=(1, 10, 1))